In [1]:
from singleton_pattern import load_dataset_loader,load_model,load_config,load_data_generator
non_dnn_method_list = load_config.get_non_dnn_method_list()
config = load_config.get_config()

model = load_model.get_model()
tensor_data_generator = load_data_generator.get_tensor_data_generator()
print(f'Method and Preprocessing Name: {config["method"]}')
print(f'Model:\n------------------\n{model}\n------------------')
test_loader = load_dataset_loader.get_test_loader()
print(f'Test Loader: {test_loader.loader_name}')

c:\Users\Bob2006\.conda\envs\compress\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Method and Preprocessing Name: GREEN
Model:
------------------
GREEN()
------------------
Test Loader: UBFC_Phys


In [2]:
import datetime
def train():
    if config['method'] in non_dnn_method_list: 
        print('non train')
        return
    start_time = datetime.datetime.now()
    train_loader = load_dataset_loader.get_train_loader()
    print(f'train_loader: {train_loader.loader_name}')
    model.train(train_loader.load_data())
    runtime = datetime.datetime.now() - start_time
    hours, remainder = divmod(runtime.total_seconds(), 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"train total time:  {int(hours)}h:{int(minutes)}m:{int(seconds)}s")
# run train
train()

non train


In [3]:
# test
model.eval()

data = test_loader.load_data()
X,y = tensor_data_generator.generate_tensor_data(data)
# print(f'Test Video Size: {data[0].shape}')

# X,y = model.preprocessing(test_df)
# pred_y = model.forward(X)

# 评价指标


Start Reading UBFC_Phys Dataset Directory...
Root Path:D:\BaiduNetdiskDownload\UBFC-PHYs_1-12


Progress: 100%|████████████████████████████████████████████████████| 12/12 [00:00<00:00, 631.60it/s]
